In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from utils.ml_utils import *

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterSampler
from tqdm import tqdm

from data_prep import load_features_data



**Load the features data, these are popular trend following features used in prior literature**

In [13]:
feats = load_features_data()
feats.tail()

ret      rVol    1d_ret   1wk_ret    1m_ret  \
date       cluster                                                     
2024-03-15 15       0.052935  0.010646 -0.009848 -0.017287  0.002016   
           16       2.355712  0.010487  0.003585  0.054587  0.082955   
           17       6.049651  0.010363  0.005988  0.013662  0.008980   
           18       1.818130  0.010196 -0.001021  0.011817  0.038944   
           19       1.014050  0.010568 -0.018105 -0.010729  0.088893   

                      1Q_ret    6M_ret   12M_ret  feature_1d_ra  \
date       cluster                                                
2024-03-15 15       0.071679  0.201208  0.202884      -0.925114   
           16       0.095509  0.074083  0.173772       0.341819   
           17       0.211590  0.529950  0.663190       0.577818   
           18      -0.010944  0.054041  0.345464      -0.100130   
           19       0.143403 -0.036800 -0.004099      -1.713252   

                    feature_1wk_ra  ...  lag5_feature_MACD_short  \
date       cluster                  ...                            
2024-03-15 15            -0.726237  ...                 0.118525   
           16             2.327878  ...                 0.464299   
           17             0.589546  ...                 0.115569   
           18             0.518333  ...                -0.049638   
           19            -0.454027  ...                 0.627858   

                    lag5_feature_MACD_medium  lag5_feature_MACD_long  \
date       cluster                                                     
2024-03-15 15                       0.407592                0.856639   
           16                       0.561470                0.464375   
           17                       0.447172                0.922814   
           18                      -0.360213               -0.198844   
           19                       0.650850                0.180584   

                    lag5_feature_skew6m  lag5_feature_skew12m  \
date       cluster                                              
2024-03-15 15                  0.066425              0.395266   
           16                  0.051510              0.388949   
           17                  0.039734              0.385207   
           18                  0.021089              0.386226   
           19                 -0.006129              0.376940   

                    lag5_feature_kurt6m  lag5_feature_kurt12m  fwd_ret1d  \
date       cluster                                                         
2024-03-15 15                  2.899242              4.029588        NaN   
           16                  2.898267              4.018791        NaN   
           17                  2.909077              4.015372        NaN   
           18                  2.867876              4.029249        NaN   
           19                  2.834308              3.977828        NaN   

                    target  targetBin  
date       cluster                     
2024-03-15 15          NaN        NaN  
           16          NaN        NaN  
           17          NaN        NaN  
           18          NaN        NaN  
           19          NaN        NaN  

[5 rows x 89 columns]

**Extract the predictive features and target which is one day forward risk adjusted returns - drop na on this subset**

In [14]:
features = [f for f in feats.columns if f.startswith("feature")]
lag_feats = [f for f in feats.columns if f.startswith("lag")]
target = ["target"]

all_feats = features + target + lag_feats
feats.dropna(subset=all_feats, inplace=True)
feats = feats[all_feats]

print("Shape of features", feats.shape)

Shape of features (117040, 79)


**Break out X and y and set up cross-validation**

In [15]:
X = feats[all_feats].copy()

baseRF = RandomForestRegressor(max_depth=5, 
                               n_estimators=1000,
                               max_features=int(1),
                               n_jobs=-3)

# simple-grid
grid = {'n_estimators': np.arange(100, 1000, 100),
        'max_depth': [3, 6, 9],
        'max_features': [int(1), 'sqrt'],
        'min_weight_fraction_leaf': np.arange(0.0, 0.05, 0.005)}

params = ParameterSampler(n_iter=25, param_distributions=grid)


**Form the training loop, here we train on 3 year expanding windows, using 90% of each split for training and 10% for tuning hyper-parameters, we then use the same model to forecast forward 3-years before re-training again.**

In [16]:
predictions = []
scores = []
for train, test in tqdm(get_cv_splits(X, split_length=252*3)):
    # break out X and y train, test
    X_train, y_train = train[features], train[target] 
    X_test, y_test = test[features], test[target]

    # hyper-param loop
    X_train2, X_val, y_train2, y_val = train_val_split(X_train, y_train)
    print(X_train2.shape, X_val.shape)

    # inner loop for parameter tuning
    gscv_scores = {'scores': [], 'grid':[]}
    for k, p in enumerate(params):
        model = RandomForestRegressor(**p)
        model.n_jobs=-1
        model.fit(X_train2, y_train2.values.reshape(y_train2.shape[0], ))
        _pred = model.predict(X_val)
        _score = mean_squared_error(y_val, _pred)
        gscv_scores['scores'].append(_score)
        gscv_scores['grid'].append(p)
        print(f'Iter: {k}: Score: {_score}')

    # now fit the best model
    best_model = pd.DataFrame(gscv_scores).sort_values(by='scores').head(1)['grid'].values[0]
    print(best_model)
    best_model = RandomForestRegressor(**best_model)
    best_model.n_jobs=-1
    best_model.fit(X_train, y_train.values.reshape(y_train.shape[0], ))
    preds = best_model.predict(X_test)

    # append the predictions
    predictions.append(pd.Series(index=y_test.index, data=preds))

    # score
    scores.append(mean_squared_error(y_test, preds))

# predictions
predictions = pd.concat(predictions).to_frame("predictions")

0it [00:00, ?it/s]

7
(13626, 13) (1500, 13)
Iter: 0: Score: 1.3397313566930171
Iter: 1: Score: 1.345525274633928
Iter: 2: Score: 1.3365385044086682
Iter: 3: Score: 1.3369046968004257
Iter: 4: Score: 1.3369205424596684
Iter: 5: Score: 1.338609732396373
Iter: 6: Score: 1.347388426348643
Iter: 7: Score: 1.3369017352714256
Iter: 8: Score: 1.3376600332812947
Iter: 9: Score: 1.3361202229643547
Iter: 10: Score: 1.3374012750602722
Iter: 11: Score: 1.3496487757544722
Iter: 12: Score: 1.3375422779642598
Iter: 13: Score: 1.3387454043328562
Iter: 14: Score: 1.3365181370161672
Iter: 15: Score: 1.3369728200207018
Iter: 16: Score: 1.3362784894960376
Iter: 17: Score: 1.338226450065164
Iter: 18: Score: 1.3362800042660212
Iter: 19: Score: 1.33907438992572
Iter: 20: Score: 1.3381081790002205
Iter: 21: Score: 1.3380336643316648
Iter: 22: Score: 1.338157449163515
Iter: 23: Score: 1.3359969748498701
Iter: 24: Score: 1.3366596936366995
{'n_estimators': 800, 'min_weight_fraction_leaf': 0.01, 'max_features': 1, 'max_depth': 3}


1it [00:25, 25.54s/it]

(27252, 13) (3020, 13)
Iter: 0: Score: 1.443540647589011
Iter: 1: Score: 1.4429349189080458
Iter: 2: Score: 1.445924156148313
Iter: 3: Score: 1.4428475822893514
Iter: 4: Score: 1.4431436464479817
Iter: 5: Score: 1.443599920929328
Iter: 6: Score: 1.4431155251252523
Iter: 7: Score: 1.4431147866468095
Iter: 8: Score: 1.443912480843022
Iter: 9: Score: 1.4428546191092047
Iter: 10: Score: 1.4484132886470036
Iter: 11: Score: 1.4428105372211835
Iter: 12: Score: 1.4437952190390737
Iter: 13: Score: 1.4425370222422258
Iter: 14: Score: 1.4425874496586506
Iter: 15: Score: 1.4443828938571692
Iter: 16: Score: 1.4427482623568788
Iter: 17: Score: 1.4434206176452844
Iter: 18: Score: 1.4439928202828232
Iter: 19: Score: 1.4430660637690778
Iter: 20: Score: 1.4625343033204263
Iter: 21: Score: 1.4468367992527802
Iter: 22: Score: 1.4432645499078882
Iter: 23: Score: 1.4439796153445787
Iter: 24: Score: 1.4449680801279583
{'n_estimators': 600, 'min_weight_fraction_leaf': 0.045, 'max_features': 1, 'max_depth': 9}

2it [01:02, 32.50s/it]

...

5it [04:19, 58.84s/it]

(81684, 13) (9060, 13)
Iter: 0: Score: 1.3419159872864066
Iter: 1: Score: 1.341994957534767
Iter: 2: Score: 1.342276999449561
Iter: 3: Score: 1.3421013103837451
Iter: 4: Score: 1.3421179453026537
Iter: 5: Score: 1.3421366642028054
Iter: 6: Score: 1.3422656064540384
Iter: 7: Score: 1.3431303112527373
Iter: 8: Score: 1.3434249942584398
Iter: 9: Score: 1.3425004226618333
Iter: 10: Score: 1.342284581090663
Iter: 11: Score: 1.3422364986604964
Iter: 12: Score: 1.3427270853471847
Iter: 13: Score: 1.342733320540898
Iter: 14: Score: 1.3422420815510656
Iter: 15: Score: 1.3421743221294884
Iter: 16: Score: 1.3419755577852146
Iter: 17: Score: 1.3423722814458972
Iter: 18: Score: 1.3421461338643188
Iter: 19: Score: 1.342591735406409
Iter: 20: Score: 1.3419262132691203
Iter: 21: Score: 1.3418827312348398
Iter: 22: Score: 1.3419525857463248
Iter: 23: Score: 1.3436988342876024
Iter: 24: Score: 1.3421160874305798
{'n_estimators': 100, 'min_weight_fraction_leaf': 0.005, 'max_features': 1, 'max_depth': 3}


6it [05:52, 70.37s/it]

(95292, 13) (10580, 13)
Iter: 0: Score: 1.305711155937452
Iter: 1: Score: 1.3057110930031801
Iter: 2: Score: 1.3059276140234957
Iter: 3: Score: 1.3060095520304322
Iter: 4: Score: 1.305607765448201
Iter: 5: Score: 1.306398197770949
Iter: 6: Score: 1.3056074680999092
Iter: 7: Score: 1.3092310560078653
Iter: 8: Score: 1.3057712477593493
Iter: 9: Score: 1.3057797846987753
Iter: 10: Score: 1.305574238472711
Iter: 11: Score: 1.3055935614363827
Iter: 12: Score: 1.3055162881384295
Iter: 13: Score: 1.3056847287230702
Iter: 14: Score: 1.305824100611432
Iter: 15: Score: 1.3060469704651052
Iter: 16: Score: 1.3118334514664716
Iter: 17: Score: 1.3055941625643053
Iter: 18: Score: 1.3058402046902113
Iter: 19: Score: 1.3055445358112618
Iter: 20: Score: 1.3061756761081125
Iter: 21: Score: 1.3057385560238517
Iter: 22: Score: 1.3076844132634111
Iter: 23: Score: 1.3104996172742118
Iter: 24: Score: 1.3055953513471776
{'n_estimators': 300, 'min_weight_fraction_leaf': 0.02, 'max_features': 'sqrt', 'max_depth'

7it [08:00, 68.70s/it]


**Now we join the forecasted returns back into the returns df, we can create a simple portfolio construction strategy where we equal weight the top half, top quartile of the clusters. We can also create a long short strategy that goes long the top quartile of forecasting cluster returns and short the bottom quartile, for a market neutral portfolio.**

In [17]:
all_feats = load_features_data()
predictions=predictions.join(all_feats[['1d_ret']])

predictions['signal_quintile']=predictions.groupby(by='date')['predictions'].apply(lambda x: pd.qcut(x, 5, labels=False)).droplevel(0)
predictions['signal_quintiles_shift'] = predictions.groupby(by='cluster')['signal_quintile'].shift(1)
(1+predictions.groupby(by=['date', 'signal_quintiles_shift'])['1d_ret'].mean().unstack()).cumprod().plot(title="Forecasted by Cluster Quintile")